### Import Libraries

In [3]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import datetime 
import datetime as dt
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import string
from nltk.corpus import wordnet
# nltk.download('stopwords')
# nltk.download('wordnet')
# import gensim
#import re
#from gensim import models, corpora

######################### Working directory #########################
import os
os.getcwd()
os.chdir("C:\\Users\\lzhga\\Documents\\Text Mining Project\\")

### Load and Merge Raw Data - Focused on SG only

In [4]:
reviews = pd.read_csv("reviews_sg.csv")

In [7]:
# reviews = reviews.drop(["cleaned_review", "cleaned_review2"], axis=1)

In [8]:
list(reviews)

['id',
 'reviews.val.date',
 'overall_review',
 'Location',
 'Title',
 'overall_rating',
 'careerOpportunities_rating',
 'compBenefits_rating',
 'cultureValues_rating',
 'seniorManagement_rating',
 'worklifeBalance_rating',
 'overview.industry',
 'header.employerName']

In [9]:
reviews.head()

,id,reviews.val.date,overall_review,Location,Title,overall_rating,careerOpportunities_rating,compBenefits_rating,cultureValues_rating,seniorManagement_rating,worklifeBalance_rating,overview.industry,header.employerName
0,44024,2020-01-01,Good benefits and flexible hours Micromanageme...,Singapore (Singapore),Consultant,3.0,3.0,3.0,3.0,3.0,4.0,Government Agencies,GovTech
1,44024,2020-01-01,"Good culture, advanced skillsets to learn from...",Singapore (Singapore),Intern,5.0,4.0,3.0,5.0,4.0,4.0,Government Agencies,GovTech
2,44024,2020-01-01,- Clean and bright offices \r\n- Friendly &amp...,Singapore (Singapore),Intern,4.0,4.0,4.0,5.0,4.0,5.0,Government Agencies,GovTech
3,44024,2019-09-21,there are many training program No management ...,Singapore (Singapore),Senior Manager,1.0,2.0,4.0,1.0,1.0,3.0,Government Agencies,GovTech
4,44054,2019-09-11,"no OT (work-life balance), just enough workloa...",Singapore (Singapore),IT Business Analyst,4.0,4.0,4.0,4.0,3.0,5.0,Healthcare Product Manufacturing,Essilor


In [10]:
reviews.dropna(subset=['overall_review'], inplace=True)
reviews['reviews.val.date'] = pd.to_datetime(reviews['reviews.val.date'], errors='coerce')
reviews['reviews.val.date'].fillna(pd.datetime(2020,1,1),inplace=True) #For recent review date, it appears as 4 weeks ago in the review date. As they have been coerced to blank, this will be replace by 1 Jan 2020
reviews['reviews.val.date'] = pd.to_datetime(reviews['reviews.val.date'], errors='coerce')

In [11]:
reviews.head()
reviews.shape

(11742, 13)

In [12]:
reviews['overview.industry'] = reviews['overview.industry'].fillna('Unknown')
reviews['header.employerName'] = reviews['header.employerName'].fillna('Unknown')

### Clean data - Remove Punctuation, tokenize, lemmatize

In [13]:
def remove_punctuation(text):
    no_punct = "".join([c for c in text if c not in string.punctuation])
    return no_punct

In [14]:
reviews['cleaned_review'] = reviews['overall_review'].apply(remove_punctuation)

In [15]:
stopwords = nltk.corpus.stopwords.words('english')
newStopWords = ['work','working','company']
stopwords.extend(newStopWords)
operators = set(('no', 'not', 'never'))
stopwords = set(stopwords) - operators

def preprocess(sentence):
    sentence = sentence.lower()
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(sentence)
    filtered_words = [w for w in tokens if not w in stopwords]
    return " ".join(filtered_words)

In [16]:
reviews['cleaned_review'] = reviews['cleaned_review'].apply(preprocess)

In [17]:
reviews.head()

,id,reviews.val.date,overall_review,Location,Title,overall_rating,careerOpportunities_rating,compBenefits_rating,cultureValues_rating,seniorManagement_rating,worklifeBalance_rating,overview.industry,header.employerName,cleaned_review
0,44024,2020-01-01,Good benefits and flexible hours Micromanageme...,Singapore (Singapore),Consultant,3.0,3.0,3.0,3.0,3.0,4.0,Government Agencies,GovTech,good benefits flexible hours micromanagement a...
1,44024,2020-01-01,"Good culture, advanced skillsets to learn from...",Singapore (Singapore),Intern,5.0,4.0,3.0,5.0,4.0,4.0,Government Agencies,GovTech,good culture advanced skillsets learn real pro...
2,44024,2020-01-01,- Clean and bright offices \r\n- Friendly &amp...,Singapore (Singapore),Intern,4.0,4.0,4.0,5.0,4.0,5.0,Government Agencies,GovTech,clean bright offices friendly amp knowledgeabl...
3,44024,2019-09-21,there are many training program No management ...,Singapore (Singapore),Senior Manager,1.0,2.0,4.0,1.0,1.0,3.0,Government Agencies,GovTech,many training program no management skill no l...
4,44054,2019-09-11,"no OT (work-life balance), just enough workloa...",Singapore (Singapore),IT Business Analyst,4.0,4.0,4.0,4.0,3.0,5.0,Healthcare Product Manufacturing,Essilor,no ot worklife balance enough workload office ...


In [18]:
# Lemmatizer
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

# Functions for lemmatizing with the appropriate POS-tag
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text)]

In [19]:
reviews['cleaned_review'] = reviews['cleaned_review'].apply(lemmatize_text)

In [20]:
reviews.head()

,id,reviews.val.date,overall_review,Location,Title,overall_rating,careerOpportunities_rating,compBenefits_rating,cultureValues_rating,seniorManagement_rating,worklifeBalance_rating,overview.industry,header.employerName,cleaned_review
0,44024,2020-01-01,Good benefits and flexible hours Micromanageme...,Singapore (Singapore),Consultant,3.0,3.0,3.0,3.0,3.0,4.0,Government Agencies,GovTech,"[good, benefit, flexible, hour, micromanagemen..."
1,44024,2020-01-01,"Good culture, advanced skillsets to learn from...",Singapore (Singapore),Intern,5.0,4.0,3.0,5.0,4.0,4.0,Government Agencies,GovTech,"[good, culture, advanced, skillsets, learn, re..."
2,44024,2020-01-01,- Clean and bright offices \r\n- Friendly &amp...,Singapore (Singapore),Intern,4.0,4.0,4.0,5.0,4.0,5.0,Government Agencies,GovTech,"[clean, bright, office, friendly, amp, knowled..."
3,44024,2019-09-21,there are many training program No management ...,Singapore (Singapore),Senior Manager,1.0,2.0,4.0,1.0,1.0,3.0,Government Agencies,GovTech,"[many, training, program, no, management, skil..."
4,44054,2019-09-11,"no OT (work-life balance), just enough workloa...",Singapore (Singapore),IT Business Analyst,4.0,4.0,4.0,4.0,3.0,5.0,Healthcare Product Manufacturing,Essilor,"[no, ot, worklife, balance, enough, workload, ..."


In [22]:
reviews_sg = reviews.copy()

In [ ]:
# Handling Negation

In [84]:
# def tag_words(sentence):
#     import re
#     # up to punctuation as in punct, put tags for words
#     # following a negative word
#     # find punctuation in the sentence
#     punct = re.findall(r'[.:;!?]',sentence)[0]
#     # create word set from sentence
#     wordSet = { x for x in re.split("[.:;!?, ]",sentence) if x }
#     keywordSet = {"don't","never", "nothing", "nowhere", "noone", "none", "not",
#                   "hasn't","hadn't","can't","couldn't","shouldn't","won't",
#                   "wouldn't","don't","doesn't","didn't","isn't","aren't","ain't"}
#     # find negative words in sentence
#     neg_words = wordSet & keywordSet
#     if neg_words:
#         for word in neg_words:
#             start_to_w = sentence[:sentence.find(word)+len(word)]
#             # put tags to words after the negative word
#             w_to_punct =  re.sub(r'\b([A-Za-z\']+)\b',r'\1_NEG',
#                                sentence[sentence.find(word)+len(word):sentence.find(punct)])
#             punct_to_end = sentence[sentence.find(punct):]
#             print(start_to_w + w_to_punct + punct_to_end)
#     else:
#         print("no negative words found ...")


In [54]:
# reviews.to_csv("reviews_final.csv", index=False)

In [ ]:
# Other methods of preprocessing
# SpaCy, TextBlob, Pattern, Stanford

# Topic modelling of reviews across companies in Singapore

In [29]:
# remove 'good' as it appears too frequently and does not contribute to the topic
# remove basic negation terms just for topic modelling
to_remove = ["good", "not", "no", "great", "well"]
reviews_sg['cleaned_review2'] = reviews_sg['cleaned_review'].apply(lambda x: [item for item in x if item not in to_remove])

In [46]:
reviews_sg.to_csv("reviews_sg_cleaned.csv", index=False)

In [85]:
reviews_sg

,id,reviews.val.date,overall_review,Location,Title,overall_rating,careerOpportunities_rating,compBenefits_rating,cultureValues_rating,seniorManagement_rating,worklifeBalance_rating,overview.industry,header.employerName,cleaned_review,cleaned_review2
0,44024,2020-01-01,Good benefits and flexible hours Micromanageme...,Singapore (Singapore),Consultant,3.0,3.0,3.0,3.0,3.0,4.0,Government Agencies,GovTech,"['good', 'benefit', 'flexible', 'hour', 'micro...","['good', 'benefit', 'flexibl', 'hour', 'microm..."
1,44024,2020-01-01,"Good culture, advanced skillsets to learn from...",Singapore (Singapore),Intern,5.0,4.0,3.0,5.0,4.0,4.0,Government Agencies,GovTech,"['good', 'culture', 'advanced', 'skillsets', '...","['good', 'cultur', 'advanc', 'skillset', 'lear..."
2,44024,2020-01-01,- Clean and bright offices \r\n- Friendly &amp...,Singapore (Singapore),Intern,4.0,4.0,4.0,5.0,4.0,5.0,Government Agencies,GovTech,"['clean', 'bright', 'office', 'friendly', 'amp...","['clean', 'bright', 'offic', 'friend', 'amp', ..."
3,44024,2019-09-21,there are many training program No management ...,Singapore (Singapore),Senior Manager,1.0,2.0,4.0,1.0,1.0,3.0,Government Agencies,GovTech,"['many', 'training', 'program', 'management', ...","['mani', 'train', 'program', 'manag', 'skill',..."
4,44054,2019-09-11,"no OT (work-life balance), just enough workloa...",Singapore (Singapore),IT Business Analyst,4.0,4.0,4.0,4.0,3.0,5.0,Healthcare Product Manufacturing,Essilor,"['ot', 'work', 'life', 'balance', 'enough', 'w...","['ot', 'work', 'life', 'balanc', 'enough', 'wo..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11737,116735,2019-06-10,"Extremely friendly people, the office team is ...",Singapore (Singapore),Full Stack Software Engineer,2.0,2.0,1.0,1.0,2.0,4.0,Unknown,Fen Associates,"['extremely', 'friendly', 'people', 'office', ...","['extrem', 'friend', 'peopl', 'offic', 'team',..."
11738,116735,2019-04-19,Many nice people to start off! There is a lot ...,Singapore (Singapore),Operations,2.0,4.0,4.0,1.0,1.0,4.0,Unknown,Fen Associates,"['many', 'nice', 'people', 'start', 'lot', 'op...","['mani', 'nice', 'peopl', 'start', 'lot', 'opp..."
11739,116735,2019-04-15,TransferWise has really cool working culture. ...,Singapore (Singapore),Senior Software Engineer,4.0,5.0,4.0,5.0,4.0,5.0,Unknown,Fen Associates,"['transferwise', 'really', 'cool', 'working', ...","['transferwis', 'realli', 'cool', 'work', 'cul..."
11740,116758,2020-01-01,Great people with lot of knowledge Very much a...,Singapore (Singapore),Staff Software Engineer,4.0,5.0,4.0,5.0,4.0,4.0,IT Services,Devoteam,"['great', 'people', 'lot', 'knowledge', 'much'...","['great', 'peopl', 'lot', 'knowledg', 'much', ..."


In [31]:
NUM_TOPICS = 4

from gensim import corpora, models

# Build LDA model for all industries
tokenized_data = reviews_sg.cleaned_review2.values.tolist() # Converts reviews to list
dictionary = corpora.Dictionary(tokenized_data) # Build a Dictionary - association word to numeric id
corpus = [dictionary.doc2bow(text) for text in tokenized_data] # Transform the collection of texts to a numerical form

lda_model = models.LdaModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary)
 
# Build the LSI model
lsi_model = models.LsiModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary)

print("LDA Model:")
 
for idx in range(NUM_TOPICS):
    # Print the first 10 most representative topics
    print("Topic #%s:" % idx, lda_model.print_topic(idx, 10))
 
print("=" * 20)
 
print("LSI Model:")
 
for idx in range(NUM_TOPICS):
    # Print the first 10 most representative topics
    print("Topic #%s:" % idx, lsi_model.print_topic(idx, 10))
 
print("=" * 20)

LDA Model:
Topic #0: 0.016*"client" + 0.014*"salary" + 0.012*"balance" + 0.012*"benefit" + 0.012*"management" + 0.009*"environment" + 0.009*"people" + 0.009*"career" + 0.008*"make" + 0.008*"recruiter"
Topic #1: 0.015*"management" + 0.010*"get" + 0.010*"employee" + 0.010*"people" + 0.009*"team" + 0.008*"lot" + 0.007*"culture" + 0.006*"opportunity" + 0.006*"new" + 0.006*"thing"
Topic #2: 0.017*"time" + 0.016*"learn" + 0.014*"lot" + 0.013*"project" + 0.011*"process" + 0.011*"team" + 0.010*"customer" + 0.010*"year" + 0.009*"get" + 0.009*"fast"
Topic #3: 0.021*"culture" + 0.014*"people" + 0.013*"office" + 0.012*"environment" + 0.012*"staff" + 0.011*"pay" + 0.010*"team" + 0.010*"colleague" + 0.009*"bad" + 0.009*"employee"
LSI Model:
Topic #0: 0.203*"management" + 0.202*"client" + 0.165*"get" + 0.160*"employee" + 0.158*"benefit" + 0.157*"team" + 0.155*"one" + 0.146*"people" + 0.139*"time" + 0.137*"office"
Topic #1: -0.408*"client" + -0.240*"sciente" + -0.217*"ensure" + -0.187*"one" + -0.177*"

In [32]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
vis

#Topics look to have quite a bit of overlap - Look to use 3 topics instead

C:\Gabriel\Udemy\Anaconda\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.025069 -0.071201       1        1  31.086063
3      0.055022 -0.108898       2        1  27.568336
0     -0.184553  0.038188       3        1  22.955603
2      0.104462  0.141911       4        1  18.389996, topic_info=    Category         Freq      Term        Total  loglift  logprob
298  Default  1462.000000    client  1462.000000  30.0000  30.0000
195  Default  1695.000000      time  1695.000000  29.0000  29.0000
616  Default   631.000000  customer   631.000000  28.0000  28.0000
19   Default  1591.000000    salary  1591.000000  27.0000  27.0000
541  Default   497.000000   sciente   497.000000  26.0000  26.0000
..       ...          ...       ...          ...      ...      ...
35    Topic4   295.635254      many  1180.913208   0.3084  -5.1824
190   Topic4   346.512970    people  2937.070557  -0.4439  -5.0236
3     Topic4   290.572845      hour  1399.608643   0.1213  -5.1997
12    Topic4   280.933533      high  1151.618652   0.2826  -5.2334
31    Topic4   265.807587    office  1695.185181  -0.1594  -5.2888

[378 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
603       1  0.452719     1
603       2  0.453984     1
603       3  0.064494     1
603       4  0.029085     1
604       1  0.402982     2
...     ...       ...   ...
158       4  0.427841  year
1656      1  0.961690  zero
1656      2  0.026714  zero
1656      3  0.006678  zero
1656      4  0.006678  zero

[895 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 1, 3])

In [47]:
# Updated to three topics since Groups 1 & 2 are too similar
NUM_TOPICS = 3

from gensim import corpora, models

# Build LDA model for all industries
tokenized_data = reviews_sg.cleaned_review2.values.tolist() # Converts reviews to list
dictionary = corpora.Dictionary(tokenized_data) # Build a Dictionary - association word to numeric id
corpus = [dictionary.doc2bow(text) for text in tokenized_data] # Transform the collection of texts to a numerical form

lda_model = models.LdaModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary)
 
# Build the LSI model
lsi_model = models.LsiModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary)

print("LDA Model:")
 
for idx in range(NUM_TOPICS):
    # Print the first 10 most representative topics
    print("Topic #%s:" % idx, lda_model.print_topic(idx, 10))
 
lda_model.save('lda_model_three.model')
print("=" * 20)
 
print("LSI Model:")
 
for idx in range(NUM_TOPICS):
    # Print the first 10 most representative topics
    print("Topic #%s:" % idx, lsi_model.print_topic(idx, 10))
 
print("=" * 20)
lsi_model.save('lsi_model_three.model')

LDA Model:
Topic #0: 0.010*"time" + 0.010*"pay" + 0.010*"client" + 0.009*"bad" + 0.009*"salary" + 0.009*"environment" + 0.009*"culture" + 0.009*"offer" + 0.009*"office" + 0.008*"team"
Topic #1: 0.013*"culture" + 0.013*"people" + 0.012*"team" + 0.009*"management" + 0.008*"learn" + 0.008*"balance" + 0.008*"get" + 0.007*"environment" + 0.007*"hour" + 0.007*"friendly"
Topic #2: 0.018*"management" + 0.014*"people" + 0.012*"lot" + 0.009*"employee" + 0.009*"staff" + 0.009*"opportunity" + 0.009*"learn" + 0.008*"environment" + 0.007*"benefit" + 0.007*"much"
LSI Model:
Topic #0: 0.203*"management" + 0.202*"client" + 0.165*"get" + 0.160*"employee" + 0.158*"benefit" + 0.157*"team" + 0.155*"one" + 0.146*"people" + 0.139*"time" + 0.137*"office"
Topic #1: 0.408*"client" + 0.240*"sciente" + 0.217*"ensure" + 0.187*"one" + 0.177*"recruiter" + -0.175*"management" + 0.147*"contractor" + 0.145*"treat" + 0.139*"service" + 0.138*"minimum"
Topic #2: 0.338*"offer" + -0.249*"management" + 0.207*"process" + 0.18

In [87]:
# For evaluation, we could use Perplexity and Coherence Scores as well.
reviews_sg["predict"] = lda_model.get_document_topics(corpus)

In [75]:
def format_topics_sentences(ldamodel=None, corpus=corpus):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

#     # Add original text to the end of the output
#     contents = pd.Series(texts)
#     sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus)

# Format
# df_dominant_topic = df_topic_sents_keywords.reset_index()
# df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
# df_dominant_topic.head(10)

In [104]:
reviews_sg

,id,reviews.val.date,overall_review,Location,Title,overall_rating,careerOpportunities_rating,compBenefits_rating,cultureValues_rating,seniorManagement_rating,worklifeBalance_rating,overview.industry,header.employerName,cleaned_review,cleaned_review2,pred_topic
0,44024,2020-01-01,Good benefits and flexible hours Micromanageme...,Singapore (Singapore),Consultant,3.0,3.0,3.0,3.0,3.0,4.0,Government Agencies,GovTech,"[good, benefit, flexible, hour, micromanagemen...","[benefit, flexible, hour, micromanagement, amo...",2
1,44024,2020-01-01,"Good culture, advanced skillsets to learn from...",Singapore (Singapore),Intern,5.0,4.0,3.0,5.0,4.0,4.0,Government Agencies,GovTech,"[good, culture, advanced, skillsets, learn, re...","[culture, advanced, skillsets, learn, real, pr...",1
2,44024,2020-01-01,- Clean and bright offices \r\n- Friendly &amp...,Singapore (Singapore),Intern,4.0,4.0,4.0,5.0,4.0,5.0,Government Agencies,GovTech,"[clean, bright, office, friendly, amp, knowled...","[clean, bright, office, friendly, amp, knowled...",0
3,44024,2019-09-21,there are many training program No management ...,Singapore (Singapore),Senior Manager,1.0,2.0,4.0,1.0,1.0,3.0,Government Agencies,GovTech,"[many, training, program, no, management, skil...","[many, training, program, management, skill, l...",1
4,44054,2019-09-11,"no OT (work-life balance), just enough workloa...",Singapore (Singapore),IT Business Analyst,4.0,4.0,4.0,4.0,3.0,5.0,Healthcare Product Manufacturing,Essilor,"[no, ot, worklife, balance, enough, workload, ...","[ot, worklife, balance, enough, workload, offi...",2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11737,116735,2019-06-10,"Extremely friendly people, the office team is ...",Singapore (Singapore),Full Stack Software Engineer,2.0,2.0,1.0,1.0,2.0,4.0,Unknown,Fen Associates,"[extremely, friendly, people, office, team, be...","[extremely, friendly, people, office, team, be...",1
11738,116735,2019-04-19,Many nice people to start off! There is a lot ...,Singapore (Singapore),Operations,2.0,4.0,4.0,1.0,1.0,4.0,Unknown,Fen Associates,"[many, nice, people, start, lot, opportunity, ...","[many, nice, people, start, lot, opportunity, ...",2
11739,116735,2019-04-15,TransferWise has really cool working culture. ...,Singapore (Singapore),Senior Software Engineer,4.0,5.0,4.0,5.0,4.0,5.0,Unknown,Fen Associates,"[transferwise, really, cool, culture, people, ...","[transferwise, really, cool, culture, people, ...",1
11740,116758,2020-01-01,Great people with lot of knowledge Very much a...,Singapore (Singapore),Staff Software Engineer,4.0,5.0,4.0,5.0,4.0,4.0,IT Services,Devoteam,"[great, people, lot, knowledge, much, agile, l...","[people, lot, knowledge, much, agile, lot, lea...",2


In [89]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
vis

C:\Gabriel\Udemy\Anaconda\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.033501 -0.118498       1        1  37.506573
2     -0.109189  0.082890       2        1  31.676033
0      0.142690  0.035608       3        1  30.817392, topic_info=    Category         Freq        Term        Total  loglift  logprob
500  Default   778.000000       offer   778.000000  30.0000  30.0000
34   Default  2906.000000  management  2906.000000  29.0000  29.0000
593  Default   979.000000         bad   979.000000  28.0000  28.0000
298  Default  1426.000000      client  1426.000000  27.0000  27.0000
366  Default   760.000000        fast   760.000000  26.0000  26.0000
..       ...          ...         ...          ...      ...      ...
17    Topic3   571.028442     project  1519.764526   0.1982  -5.0404
228   Topic3   621.418945    employee  2054.211426  -0.0186  -4.9558
30    Topic3   629.619934         lot  2207.650635  -0.0775  -4.9427
149   Topic3   489.593658        need  1316.526245   0.1879  -5.1942
1     Topic3   517.365295     benefit  1810.375977  -0.0754  -5.1390

[300 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
603       1  0.609720      1
603       2  0.045634      1
603       3  0.343522      1
604       1  0.563712      2
604       2  0.040832      2
...     ...       ...    ...
509       2  0.006665  worry
509       3  0.986482  worry
158       1  0.252614   year
158       2  0.180559   year
158       3  0.566261   year

[643 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 1])

In [112]:
reviews_sg["pred_topic"] = df_topic_sents_keywords

In [113]:
reviews_sg["pred_topic"].value_counts()

1.0    4904
2.0    3654
0.0    3184
Name: pred_topic, dtype: int64

In [114]:
reviews_sg["pred_topic"] = reviews_sg["pred_topic"].map({0: "Culture/Team", 1: "Management", 2: "Benefits/Renumeration"})

# Topic #0: Circle 1 = Culture/Team
# Topic #1: Circle 2 = Management
# Topic #2: Circle 3 = Benefits/Renumeration

In [115]:
reviews_sg

,id,reviews.val.date,overall_review,Location,Title,overall_rating,careerOpportunities_rating,compBenefits_rating,cultureValues_rating,seniorManagement_rating,worklifeBalance_rating,overview.industry,header.employerName,cleaned_review,cleaned_review2,pred_topic
0,44024,2020-01-01,Good benefits and flexible hours Micromanageme...,Singapore (Singapore),Consultant,3.0,3.0,3.0,3.0,3.0,4.0,Government Agencies,GovTech,"[good, benefit, flexible, hour, micromanagemen...","[benefit, flexible, hour, micromanagement, amo...",Benefits/Renumeration
1,44024,2020-01-01,"Good culture, advanced skillsets to learn from...",Singapore (Singapore),Intern,5.0,4.0,3.0,5.0,4.0,4.0,Government Agencies,GovTech,"[good, culture, advanced, skillsets, learn, re...","[culture, advanced, skillsets, learn, real, pr...",Management
2,44024,2020-01-01,- Clean and bright offices \r\n- Friendly &amp...,Singapore (Singapore),Intern,4.0,4.0,4.0,5.0,4.0,5.0,Government Agencies,GovTech,"[clean, bright, office, friendly, amp, knowled...","[clean, bright, office, friendly, amp, knowled...",Culture/Team
3,44024,2019-09-21,there are many training program No management ...,Singapore (Singapore),Senior Manager,1.0,2.0,4.0,1.0,1.0,3.0,Government Agencies,GovTech,"[many, training, program, no, management, skil...","[many, training, program, management, skill, l...",Management
4,44054,2019-09-11,"no OT (work-life balance), just enough workloa...",Singapore (Singapore),IT Business Analyst,4.0,4.0,4.0,4.0,3.0,5.0,Healthcare Product Manufacturing,Essilor,"[no, ot, worklife, balance, enough, workload, ...","[ot, worklife, balance, enough, workload, offi...",Benefits/Renumeration
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11737,116735,2019-06-10,"Extremely friendly people, the office team is ...",Singapore (Singapore),Full Stack Software Engineer,2.0,2.0,1.0,1.0,2.0,4.0,Unknown,Fen Associates,"[extremely, friendly, people, office, team, be...","[extremely, friendly, people, office, team, be...",Management
11738,116735,2019-04-19,Many nice people to start off! There is a lot ...,Singapore (Singapore),Operations,2.0,4.0,4.0,1.0,1.0,4.0,Unknown,Fen Associates,"[many, nice, people, start, lot, opportunity, ...","[many, nice, people, start, lot, opportunity, ...",Benefits/Renumeration
11739,116735,2019-04-15,TransferWise has really cool working culture. ...,Singapore (Singapore),Senior Software Engineer,4.0,5.0,4.0,5.0,4.0,5.0,Unknown,Fen Associates,"[transferwise, really, cool, culture, people, ...","[transferwise, really, cool, culture, people, ...",Management
11740,116758,2020-01-01,Great people with lot of knowledge Very much a...,Singapore (Singapore),Staff Software Engineer,4.0,5.0,4.0,5.0,4.0,4.0,IT Services,Devoteam,"[great, people, lot, knowledge, much, agile, l...","[people, lot, knowledge, much, agile, lot, lea...",Benefits/Renumeration


In [116]:
reviews_sg.to_csv("reviews_final_modelled.csv", index=False)

### Topic Modelling - Breakdown by employers
Alternatively, we could just filter by respective industries.

In [35]:
reviews_sg['header.employerName'].value_counts().head(20)

Sciente                 294
Singtel                 228
Amazon                  164
DBS Bank                160
Citi                    112
Unknown                 111
Facebook                111
Robert Walters           97
Hays                     92
Citibank                 92
Apple                    80
J.P. Morgan              79
Dell                     79
Aryan Search Pte Ltd     72
Visa Inc.                70
Shopee                   60
Oracle                   56
Lazada                   56
Vital                    53
GO-JEK                   52
Name: header.employerName, dtype: int64

In [45]:
reviews_sg["overview.industry"].value_counts() # Itll be interesting to look at topic modelling across industries as well

Unknown                         2460
IT Services                     1247
Computer Hardware & Software    1076
Internet                         817
Staffing & Outsourcing           566
                                ... 
Sporting Goods Shops               1
Vehicle Dealers                    1
Local Councils                     1
Preschool & Child Care             1
Mining                             1
Name: overview.industry, Length: 89, dtype: int64

In [36]:
reviews_sg_ecommerce = reviews_sg[(reviews_sg['header.employerName']=='Amazon')|(reviews_sg['header.employerName']=='Shopee')|(reviews_sg['header.employerName']=='Lazada')]
reviews_sg_banks = reviews_sg[(reviews_sg['header.employerName']=='DBS Bank')|(reviews_sg['header.employerName']=='Citibank')|(reviews_sg['header.employerName']=='Citi')|(reviews_sg['header.employerName']=='J.P. Morgan')]
reviews_sg_recruitment = reviews_sg[(reviews_sg['header.employerName']=='Hays')|(reviews_sg['header.employerName']=='Robert Walters')|(reviews_sg['header.employerName']=='Aryan Search Pte Ltd')]

In [37]:
print(reviews_sg_ecommerce.shape)
print(reviews_sg_banks.shape)
print(reviews_sg_recruitment.shape)

(280, 15)
(443, 15)
(261, 15)


### Ecommerce

In [39]:
# ecommerce

NUM_TOPICS = 3

from gensim import corpora, models

# Build LDA model for all industries
tokenized_data = reviews_sg_ecommerce.cleaned_review2.values.tolist() # Converts reviews to list
dictionary = corpora.Dictionary(tokenized_data) # Build a Dictionary - association word to numeric id
corpus = [dictionary.doc2bow(text) for text in tokenized_data] # Transform the collection of texts to a numerical form

lda_model_ecommerce = models.LdaModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary)
 
# Build the LSI model
lsi_model_ecommerce = models.LsiModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary)

print("LDA Model:")
 
for idx in range(NUM_TOPICS):
    # Print the first 10 most representative topics
    print("Topic #%s:" % idx, lda_model.print_topic(idx, 10))
 
print("=" * 20)
 
print("LSI Model:")
 
for idx in range(NUM_TOPICS):
    # Print the first 10 most representative topics
    print("Topic #%s:" % idx, lsi_model.print_topic(idx, 10))
 
print("=" * 20)


LDA Model:
Topic #0: 0.014*"management" + 0.012*"team" + 0.010*"staff" + 0.009*"time" + 0.009*"employee" + 0.008*"get" + 0.007*"offer" + 0.007*"office" + 0.007*"many" + 0.007*"people"
Topic #1: 0.017*"culture" + 0.015*"client" + 0.014*"benefit" + 0.012*"people" + 0.010*"one" + 0.010*"pay" + 0.009*"lot" + 0.008*"bad" + 0.008*"career" + 0.008*"customer"
Topic #2: 0.014*"learn" + 0.014*"environment" + 0.012*"people" + 0.012*"culture" + 0.011*"management" + 0.010*"lot" + 0.009*"team" + 0.009*"hour" + 0.008*"employee" + 0.008*"project"
LSI Model:
Topic #0: 0.203*"management" + 0.202*"client" + 0.165*"get" + 0.160*"employee" + 0.158*"benefit" + 0.157*"team" + 0.155*"one" + 0.146*"people" + 0.139*"time" + 0.137*"office"
Topic #1: 0.408*"client" + 0.240*"sciente" + 0.217*"ensure" + 0.187*"one" + 0.177*"recruiter" + -0.175*"management" + 0.147*"contractor" + 0.145*"treat" + 0.139*"service" + 0.138*"minimum"
Topic #2: 0.338*"offer" + -0.249*"management" + 0.207*"process" + 0.189*"office" + 0.169

In [40]:
vis_ecommerce = pyLDAvis.gensim.prepare(lda_model_ecommerce, corpus, dictionary)
vis_ecommerce

# Topics tend to be around team/culture (1), Overall environment (2), Management/hours/worklife balance (3)
# Could do further data cleaning + scrapping over a longer period of data to get better separation of groupings.

C:\Gabriel\Udemy\Anaconda\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.125767 -0.027870       1        1  40.932228
2     -0.096760 -0.063667       2        1  33.783024
1     -0.029007  0.091537       3        1  25.284746, topic_info=    Category       Freq      Term      Total  loglift  logprob
5    Default  31.000000      long  31.000000  30.0000  30.0000
6    Default  65.000000       lot  65.000000  29.0000  29.0000
4    Default  42.000000     learn  42.000000  28.0000  28.0000
72   Default  29.000000      tech  29.000000  27.0000  27.0000
158  Default  32.000000        go  32.000000  26.0000  26.0000
..       ...        ...       ...        ...      ...      ...
73    Topic3  14.095938  worklife  33.046219   0.5229  -4.6064
31    Topic3  17.620300       new  75.068588  -0.0744  -4.3833
8     Topic3  13.641877      team  34.430271   0.4492  -4.6392
69    Topic3  14.321755   project  53.884945   0.0499  -4.5906
65    Topic3  10.556394  industry  17.295557   0.8813  -4.8956

[191 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
81        1  0.973675         2
45        1  0.937803       ali
59        1  0.175526    allows
59        2  0.087763    allows
59        3  0.789868    allows
...     ...       ...       ...
138       2  0.067418   welfare
138       3  0.876430   welfare
73        1  0.090782  worklife
73        2  0.484170  worklife
73        3  0.423649  worklife

[346 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2])

### Banks

In [42]:
# Banks

NUM_TOPICS = 3

from gensim import corpora, models

# Build LDA model for all industries
tokenized_data = reviews_sg_banks.cleaned_review2.values.tolist() # Converts reviews to list
dictionary = corpora.Dictionary(tokenized_data) # Build a Dictionary - association word to numeric id
corpus = [dictionary.doc2bow(text) for text in tokenized_data] # Transform the collection of texts to a numerical form

lda_model_banks = models.LdaModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary)
 
# Build the LSI model
lsi_model_banks = models.LsiModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary)

print("LDA Model:")
 
for idx in range(NUM_TOPICS):
    # Print the first 10 most representative topics
    print("Topic #%s:" % idx, lda_model.print_topic(idx, 10))
 
print("=" * 20)
 
print("LSI Model:")
 
for idx in range(NUM_TOPICS):
    # Print the first 10 most representative topics
    print("Topic #%s:" % idx, lsi_model.print_topic(idx, 10))
 
print("=" * 20)



LDA Model:
Topic #0: 0.014*"management" + 0.012*"team" + 0.010*"staff" + 0.009*"time" + 0.009*"employee" + 0.008*"get" + 0.007*"offer" + 0.007*"office" + 0.007*"many" + 0.007*"people"
Topic #1: 0.017*"culture" + 0.015*"client" + 0.014*"benefit" + 0.012*"people" + 0.010*"one" + 0.010*"pay" + 0.009*"lot" + 0.008*"bad" + 0.008*"career" + 0.008*"customer"
Topic #2: 0.014*"learn" + 0.014*"environment" + 0.012*"people" + 0.012*"culture" + 0.011*"management" + 0.010*"lot" + 0.009*"team" + 0.009*"hour" + 0.008*"employee" + 0.008*"project"
LSI Model:
Topic #0: 0.203*"management" + 0.202*"client" + 0.165*"get" + 0.160*"employee" + 0.158*"benefit" + 0.157*"team" + 0.155*"one" + 0.146*"people" + 0.139*"time" + 0.137*"office"
Topic #1: 0.408*"client" + 0.240*"sciente" + 0.217*"ensure" + 0.187*"one" + 0.177*"recruiter" + -0.175*"management" + 0.147*"contractor" + 0.145*"treat" + 0.139*"service" + 0.138*"minimum"
Topic #2: 0.338*"offer" + -0.249*"management" + 0.207*"process" + 0.189*"office" + 0.169

In [43]:
vis_banks = pyLDAvis.gensim.prepare(lda_model_banks, corpus, dictionary)
vis_banks

C:\Gabriel\Udemy\Anaconda\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.150564 -0.019671       1        1  40.247486
2      0.093999 -0.108846       2        1  33.004883
1      0.056566  0.128517       3        1  26.747635, topic_info=    Category       Freq          Term      Total  loglift  logprob
95   Default  72.000000         staff  72.000000  30.0000  30.0000
68   Default  59.000000        change  59.000000  29.0000  29.0000
74   Default  62.000000           hit  62.000000  28.0000  28.0000
83   Default  71.000000        people  71.000000  27.0000  27.0000
90   Default  32.000000     implement  32.000000  26.0000  26.0000
..       ...        ...           ...        ...      ...      ...
5     Topic3  13.578150      employee  30.231852   0.5183  -4.8293
441   Topic3  11.275875          year  20.099831   0.7407  -5.0151
25    Topic3  11.918625        decent  23.907011   0.6227  -4.9597
24    Topic3  13.941270  compensation  44.598923   0.1559  -4.8029
125   Topic3  11.332816         learn  35.656731   0.1725  -5.0101

[156 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
52        1  0.969308   agile
67        1  0.034413    alot
67        2  0.963561    alot
67        3  0.034413    alot
15        1  0.061604  always
...     ...       ...     ...
553       1  0.932723   world
553       2  0.077727   world
441       1  0.199007    year
441       2  0.248758    year
441       3  0.547268    year

[280 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2])